In [46]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

In [47]:
# file_name = "CSE_Netflow.csv"
file_name = "NF-CSE-CIC-IDS2018-v2.parquet"
data = pd.read_parquet(file_name)

In [48]:
data.Label.value_counts()

Label
0    16635567
1     2258141
Name: count, dtype: int64

In [49]:
data.groupby(by='Attack').count()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,16635567,16635567,16635567,16635567,16635567,16635567,16635567,16635567,16635567,16635567,...,16635567,16635567,16635567,16635567,16635567,16635567,16635567,16635567,16635567,16635567
Bot,143097,143097,143097,143097,143097,143097,143097,143097,143097,143097,...,143097,143097,143097,143097,143097,143097,143097,143097,143097,143097
Brute Force -Web,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,...,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143
Brute Force -XSS,927,927,927,927,927,927,927,927,927,927,...,927,927,927,927,927,927,927,927,927,927
DDOS attack-HOIC,1080858,1080858,1080858,1080858,1080858,1080858,1080858,1080858,1080858,1080858,...,1080858,1080858,1080858,1080858,1080858,1080858,1080858,1080858,1080858,1080858
DDOS attack-LOIC-UDP,2112,2112,2112,2112,2112,2112,2112,2112,2112,2112,...,2112,2112,2112,2112,2112,2112,2112,2112,2112,2112
DDoS attacks-LOIC-HTTP,307300,307300,307300,307300,307300,307300,307300,307300,307300,307300,...,307300,307300,307300,307300,307300,307300,307300,307300,307300,307300
DoS attacks-GoldenEye,27723,27723,27723,27723,27723,27723,27723,27723,27723,27723,...,27723,27723,27723,27723,27723,27723,27723,27723,27723,27723
DoS attacks-Hulk,432648,432648,432648,432648,432648,432648,432648,432648,432648,432648,...,432648,432648,432648,432648,432648,432648,432648,432648,432648,432648


In [50]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [51]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [52]:
data.Attack.unique()

array(['SSH-Bruteforce', 'Benign', 'DDoS attacks-LOIC-HTTP',
       'DDOS attack-HOIC', 'DoS attacks-Slowloris', 'DoS attacks-Hulk',
       'FTP-BruteForce', 'Infilteration', 'Bot', 'DoS attacks-GoldenEye',
       'Brute Force -Web', 'DoS attacks-SlowHTTPTest', 'SQL Injection',
       'DDOS attack-LOIC-UDP', 'Brute Force -XSS'], dtype=object)

In [ ]:
# Scale the dataset based on your needs (machine capacity) ideally ~500k rows

data = data.groupby(by='Attack').sample(frac=0.02, random_state=13)
# data_attack = data[data['Label'] == 1]
# data_benign = data[data['Label'] == 0].sample(frac=0.1, random_state=13)
# data = pd.concat([data_attack, data_benign], axis=0)
# data = data.sample(frac=0.1, random_state=13).reset_index(drop=True)
data.Label.value_counts()

Label
0    332711
1     45163
Name: count, dtype: int64

In [54]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Benign,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711,...,332711,332711,332711,332711,332711,332711,332711,332711,332711,332711
Bot,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862,...,2862,2862,2862,2862,2862,2862,2862,2862,2862,2862
Brute Force -Web,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
Brute Force -XSS,19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
DDOS attack-HOIC,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617,...,21617,21617,21617,21617,21617,21617,21617,21617,21617,21617
DDOS attack-LOIC-UDP,42,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
DDoS attacks-LOIC-HTTP,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146,...,6146,6146,6146,6146,6146,6146,6146,6146,6146,6146
DoS attacks-GoldenEye,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
DoS attacks-Hulk,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653,...,8653,8653,8653,8653,8653,8653,8653,8653,8653,8653


In [55]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)

In [56]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [57]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [58]:
# (Modified)
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()
X_train['id'] = X_train.index

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()
X_test['id'] = X_test.index

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [59]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h,id
9910620,172.31.65.112,172.217.15.65,6.210352e-09,1.019912e-10,0.000043,5.354863e-07,0.000117,4.094895e-07,1.624410e-08,1.157705e-08,...,0.000258,0.001348,3.704528e-09,3.704528e-09,6.190550e-09,6.190550e-09,0.000000,3.764741e-09,"[6.210351872620525e-09, 1.019912495558199e-10,...",9910620
17557027,18.218.11.51,172.31.69.28,4.579454e-08,1.543672e-07,0.000122,1.161360e-06,0.000266,1.161360e-06,1.197824e-07,8.536806e-08,...,0.015222,0.006244,2.731684e-08,2.731684e-08,4.564852e-08,4.564852e-08,0.000000,2.776084e-08,"[4.57945440200515e-08, 1.5436721751920067e-07,...",17557027
4328285,190.38.79.218,172.31.64.101,4.079400e-08,1.778170e-09,0.000103,1.034545e-06,0.000071,8.276362e-07,1.644264e-09,2.215507e-09,...,0.003398,0.001695,2.433397e-08,2.433397e-08,4.066392e-08,4.066392e-08,0.000000,2.472949e-08,"[4.079399803822932e-08, 1.778169602167593e-09,...",4328285
1287389,172.31.68.15,172.31.0.2,6.533832e-09,5.154645e-09,0.000038,5.997981e-07,0.000067,5.997981e-07,6.565366e-09,6.565366e-09,...,0.000000,0.000000,7.054050e-08,7.054050e-08,6.235997e-08,5.346222e-09,0.000036,7.168706e-08,"[6.5338315603211064e-09, 5.154645331031908e-09...",1287389
4527816,172.31.66.21,172.31.0.2,1.168657e-08,9.219721e-09,0.000079,1.072813e-06,0.000097,1.072813e-06,1.174297e-08,1.174297e-08,...,0.000000,0.000000,1.261704e-07,1.261704e-07,1.115385e-07,9.562381e-09,0.000064,1.282212e-07,"[1.1686566408769841e-08, 9.219721141969458e-09...",4527816


In [60]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [ ]:
# Training graph (Modified)

train['id'] = train.index

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

test['id'] = test.index

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack", "id"], create_using=nx.MultiGraph())
# print(test_g)
test_g = test_g.to_directed()
# print(test_g)
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label', "id"])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [ ]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [ ]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [ ]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [ ]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [ ]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [ ]:
# Convert to GPU
train_g = train_g.to('cuda')

In [ ]:
cnt_wait = 0
best = 1e9
best_t = 0
dur = []
node_features = train_g.ndata['h'] 
edge_features = train_g.edata['h']

for epoch in range(epochs):
    dgi.train()
    if epoch >= 3:
        t0 = time.time()

    dgi_optimizer.zero_grad()
    loss = dgi(train_g, node_features, edge_features)
    loss.backward()
    dgi_optimizer.step()

    if loss < best:
        best = loss
        best_t = epoch
        cnt_wait = 0
        torch.save(dgi.state_dict(), 'best_dgi_CSE_v2.pkl')
    else:
        cnt_wait += 1

  # if cnt_wait == patience:
  #     print('Early stopping!')
  #     break

    if epoch >= 3:
        dur.append(time.time() - t0)

    if epoch % 50 == 0:

        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
            "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
              loss.item(),
              train_g.num_edges() / np.mean(dur) / 1000))

In [26]:
dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))

/tmp/ipykernel_26694/1688994637.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_CSE_v2.pkl'))


<All keys matched successfully>

In [27]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [28]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [29]:
# Convert to GPU
test_g = test_g.to('cuda')

In [30]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [31]:
# Multimodal (Fusion) Learning

df_train = pd.DataFrame(training_emb,)
# map the id to the original data
df_train['id'] = train_g.edata['id'].detach().cpu().numpy()


df_raw_train = pd.DataFrame(X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_fuse_train = pd.merge(df_train, df_raw_train, on='id', how='left')
df_fuse_train = df_fuse_train.drop(columns=["id"])
df_fuse_train["Attacks"] = train_g.edata['Attack'].detach().cpu().numpy()
df_fuse_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb,)
# map the id to the original data
df_test['id'] = test_g.edata['id'].detach().cpu().numpy()

df_raw_test = pd.DataFrame(X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h"]))
df_raw_test = pd.merge(df_test, df_raw_test, on='id', how='left')
df_fuse_test = df_raw_test.drop(columns=["id"])
df_fuse_test["Attacks"] = test_g.edata['Attack'].detach().cpu().numpy()
df_fuse_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

# Embeddings CBLOF  Embeddings

In [32]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
from pyod.models.cblof import CBLOF
import gc

from tqdm import tqdm
import itertools

In [33]:
benign_fuse_train_samples = df_fuse_train[df_fuse_train.Label == 0].drop(columns=["Label", "Attacks"])
normal_fuse_train_samples = df_fuse_train.drop(columns=["Label", "Attacks"])

fuse_train_labels = df_fuse_train["Label"]
fuse_test_labels = df_fuse_test["Label"]

fuse_test_samples = df_fuse_test.drop(columns=["Label", "Attacks"])

In [33]:
fuse_test_samples

,0,1,2,3,4,5,6,7,8,9,...,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE
0,-0.065264,0.141264,0.006424,0.109434,0.149380,-0.002307,0.054591,0.073480,0.162122,0.157876,...,9.947912e-08,0.000000e+00,0.000815,0.002674,5.697104e-08,5.697104e-08,7.056873e-08,7.056873e-08,0.0,5.731920e-08
1,-0.065264,0.141264,0.006424,0.109434,0.149380,-0.002307,0.054591,0.073480,0.162122,0.157876,...,9.834285e-08,0.000000e+00,0.000806,0.002644,5.632030e-08,5.632030e-08,6.976268e-08,6.976268e-08,0.0,5.666449e-08
2,-0.065264,0.141264,0.006424,0.109434,0.149380,-0.002307,0.054591,0.073480,0.162122,0.157876,...,9.948407e-08,0.000000e+00,0.000815,0.002674,5.697387e-08,5.697387e-08,7.057224e-08,7.057224e-08,0.0,5.732205e-08
3,-0.065264,0.141264,0.006424,0.109434,0.149380,-0.002307,0.054591,0.073480,0.162122,0.157876,...,9.947826e-08,0.000000e+00,0.000815,0.002674,5.697055e-08,5.697055e-08,7.056812e-08,7.056812e-08,0.0,5.731871e-08
4,-0.065264,0.141264,0.006424,0.109434,0.149380,-0.002307,0.054591,0.073480,0.162122,0.157876,...,9.948407e-08,0.000000e+00,0.000815,0.002674,5.697388e-08,5.697388e-08,7.057224e-08,7.057224e-08,0.0,5.732205e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235125,-0.021808,-0.025892,0.180949,0.047172,0.121170,0.099557,0.064873,0.177384,0.085333,-0.003861,...,0.000000e+00,0.000000e+00,0.025841,0.000000,1.789069e-06,1.789069e-06,2.216079e-06,2.216079e-06,0.0,1.800003e-06
235126,-0.057339,-0.015571,0.208725,0.030920,0.098021,0.099891,0.055402,0.196728,0.098082,-0.046024,...,0.000000e+00,0.000000e+00,0.010997,0.005485,3.834516e-07,3.834516e-07,4.749727e-07,4.749727e-07,0.0,3.857949e-07
235127,-0.057188,-0.026571,0.210925,0.032160,0.099595,0.103495,0.055317,0.188349,0.093165,-0.051476,...,0.000000e+00,0.000000e+00,0.011180,0.005485,3.834508e-07,3.834508e-07,4.749717e-07,4.749717e-07,0.0,3.857941e-07
235128,-0.049498,-0.006132,0.184719,0.031591,0.085903,0.082838,0.063280,0.197226,0.120594,-0.037659,...,6.958279e-08,4.638853e-08,0.000190,0.000677,8.881348e-09,8.881348e-09,1.645360e-08,1.645360e-08,0.0,1.336438e-08


In [61]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [62]:
n_est = [5,6,7,9,10] # cant be lower than 5 or 4
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(benign_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [01:46<00:00,  3.56s/it]

{'n_estimators': 6, 'con': 0.05}
0.9352821093560512
              precision    recall  f1-score   support

           0     0.9021    0.9532    0.9269     99552
           1     0.9641    0.9240    0.9436    135578

    accuracy                         0.9364    235130
   macro avg     0.9331    0.9386    0.9353    235130
weighted avg     0.9379    0.9364    0.9366    235130



In [63]:
n_est = [5,6,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(normal_fuse_train_samples)
    except Exception as e:
        print(n_est)
        continue
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 30/30 [04:56<00:00,  9.87s/it]

{'n_estimators': 10, 'con': 0.2}
0.5270590016541896
              precision    recall  f1-score   support

           0     0.4822    0.8993    0.6278     99552
           1     0.7974    0.2909    0.4263    135578

    accuracy                         0.5485    235130
   macro avg     0.6398    0.5951    0.5271    235130
weighted avg     0.6639    0.5485    0.5116    235130



In [64]:
# HBOS  Embeddings+Raw (Multimodal)

In [65]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [ ]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

 47%|████▋     | 17/36 [00:59<01:08,  3.62s/it]

In [ ]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:52<00:00,  1.46s/it]

{'n_estimators': 5, 'con': 0.01}
0.4893857068710194
              precision    recall  f1-score   support

           0     0.6379    0.6315    0.6347     38196
           1     0.3410    0.3472    0.3441     20972

    accuracy                         0.5307     59168
   macro avg     0.4895    0.4894    0.4894     59168
weighted avg     0.5327    0.5307    0.5317     59168



In [ ]:
##  PCA  Emb+Raw (Multimodal/Fusion) Learning

In [ ]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:50<00:00,  1.41s/it]

{'n_estimators': 25, 'con': 0.04}
0.4521499568535427
              precision    recall  f1-score   support

           0     0.6238    0.3855    0.4765     38196
           1     0.3400    0.5766    0.4278     20972

    accuracy                         0.4532     59168
   macro avg     0.4819    0.4810    0.4521     59168
weighted avg     0.5232    0.4532    0.4592     59168



In [ ]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = y_pred

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 36/36 [01:01<00:00,  1.72s/it]

{'n_estimators': 5, 'con': 0.05}
0.43776217172464377
              precision    recall  f1-score   support

           0     0.5956    0.5155    0.5526     38196
           1     0.2911    0.3624    0.3229     20972

    accuracy                         0.4612     59168
   macro avg     0.4433    0.4390    0.4378     59168
weighted avg     0.4877    0.4612    0.4712     59168



In [ ]:
##  IF  Emb+Raw

In [ ]:
for i in range(len(benign_fuse_train_samples.columns)):
    benign_fuse_train_samples.rename(columns={benign_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(normal_fuse_train_samples.columns)):
    normal_fuse_train_samples.rename(columns={normal_fuse_train_samples.columns[i]: f"feature {i}"}, inplace=True)

for i in range(len(fuse_test_samples.columns)):
    fuse_test_samples.rename(columns={fuse_test_samples.columns[i]: f"feature {i}"}, inplace=True)

In [ ]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_fuse_train_samples.to_numpy())
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]

/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  4%|▍         | 1/24 [00:00<00:11,  2.09it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
  8%|▊         | 2/24 [00:00<00:09,  2.30it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warnings.warn(
 12%|█▎        | 3/24 [00:01<00:08,  2.37it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but IsolationForest was fitted without feature names
  warni

{'n_estimators': 100, 'con': 0.01}
0.39985799202747224
              precision    recall  f1-score   support

           0     0.6452    0.9885    0.7807     38196
           1     0.3183    0.0098    0.0190     20972

    accuracy                         0.6416     59168
   macro avg     0.4817    0.4991    0.3999     59168
weighted avg     0.5293    0.6416    0.5107     59168



In [ ]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_fuse_train_samples)
    y_pred = clf_if.predict(fuse_test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(fuse_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(fuse_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:20<00:00,  1.15it/s]

{'n_estimators': 150, 'con': 0.01}
0.4000915949078085
              precision    recall  f1-score   support

           0     0.6445    0.9844    0.7790     38196
           1     0.2797    0.0110    0.0212     20972

    accuracy                         0.6394     59168
   macro avg     0.4621    0.4977    0.4001     59168
weighted avg     0.5152    0.6394    0.5104     59168



## Supervised attack classification (Fusion)

We now train a supervised classifier on the fused features to predict multi-class attack labels:
- Features: embeddings + raw numeric features from `df_fuse_train`/`df_fuse_test` (without `Label`, `Attacks`).
- Target: `Attacks` (encoded integer classes from earlier `LabelEncoder`).
- Model: HistGradientBoostingClassifier (fast, strong on tabular data). Class imbalance handled via per-sample weights.
- Metrics: macro F1, per-class report, and confusion matrix.

In [34]:
# Prepare supervised train/test for attack classification
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils import class_weight
import pickle

# Build train features/targets from already prepared fused DataFrames
X_sup_train = df_fuse_train.drop(columns=["Label", "Attacks"]).copy()
y_sup_train = df_fuse_train["Attacks"].copy()

X_sup_test = df_fuse_test.drop(columns=["Label", "Attacks"]).copy()
y_sup_test = df_fuse_test["Attacks"].copy()

# Compute sample weights to mitigate class imbalance on training set
classes = np.unique(y_sup_train)
class_w = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes, y=y_sup_train
)
class_to_w = {c: w for c, w in zip(classes, class_w)}
sample_weight = y_sup_train.map(class_to_w).values

# Feature names to all str
X_sup_train.columns = X_sup_train.columns.map(str)
X_sup_test.columns = X_sup_test.columns.map(str)

## Classification Methods (Fused Features)

Now we'll compare multiple classification algorithms on the fused features:

In [35]:
# Random Forest with Grid Search
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]
    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            rf_clf = RandomForestClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            rf_clf.fit(X_sup_train, y_sup_train)
            y_pred_rf = rf_clf.predict(X_sup_test)
            rf_f1 = f1_score(y_sup_test, y_pred_rf, average='macro')
            
            if rf_f1 > score:
                score = rf_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = rf_clf
                # Save best model
                with open('best_rf_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(rf_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_rf_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Random Forest: {str(e)}")

RANDOM FOREST CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [14:55<00:00, 99.50s/it] 



Best Parameters: {'n_estimators': 200, 'max_depth': 30}
Best Macro F1: 0.7520
Model saved to: best_rf_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.9323    0.9943    0.9623     99552
           1     1.0000    0.8857    0.9394      8524
           2     0.0224    0.4531    0.0427       128
           3     0.0203    0.2143    0.0371        56
           4     0.9986    0.9488    0.9731     65118
           5     0.9508    1.0000    0.9748       116
           6     0.9999    1.0000    0.9999     18324
           7     1.0000    1.0000    1.0000      1668
           8     1.0000    1.0000    1.0000     26008
           9     1.0000    1.0000    1.0000       856
          10     1.0000    0.9910    0.9955       554
          11     1.0000    1.0000    1.0000      1518
          12     0.8666    0.5192    0.6494      7018
          13     0.0216    0.2143    0.0392        28
          14     1.0000    0.5000    0.6667      5662

    accur

In [36]:
# XGBoost with Grid Search
try:
    from xgboost import XGBClassifier
    import pickle
    
    print("="*60)
    print("XGBOOST CLASSIFIER (Fused Features)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        try:
            xgb_clf = XGBClassifier(
                n_estimators=n_est,
                max_depth=depth,
                learning_rate=lr,
                random_state=13,
                tree_method='hist', 
                device="cuda"
            )
            
            xgb_clf.fit(X_sup_train, y_sup_train, sample_weight=sample_weight)
            y_pred_xgb = xgb_clf.predict(X_sup_test)
            xgb_f1 = f1_score(y_sup_test, y_pred_xgb, average='macro')
            
            if xgb_f1 > score:
                score = xgb_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth,
                    'learning_rate': lr
                }
                best_model = xgb_clf
                # Save best model
                xgb_clf.save_model('best_xgb_classifier_fused.json')
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_xgb_classifier_fused.json\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed. XGBoost might not support your GPU.")
    
except ImportError:
    print("XGBoost not installed. Install with: pip install xgboost")
except Exception as e:
    print(f"Unexpected error: {str(e)}")

XGBOOST CLASSIFIER (Fused Features)
Testing 27 configurations...


  0%|          | 0/27 [00:00<?, ?it/s]/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [19:51:17] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
100%|██████████| 27/27 [32:18<00:00, 71.80s/it] 



Best Parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.7172
Model saved to: best_xgb_classifier_fused.json

              precision    recall  f1-score   support

           0     0.8674    0.9934    0.9261     99552
           1     1.0000    0.9999    0.9999      8524
           2     0.0235    0.2891    0.0434       128
           3     0.0190    0.1429    0.0335        56
           4     0.9976    0.9661    0.9816     65118
           5     0.9508    1.0000    0.9748       116
           6     0.9996    1.0000    0.9998     18324
           7     0.9982    1.0000    0.9991      1668
           8     0.9999    0.6658    0.7993     26008
           9     1.0000    1.0000    1.0000       856
          10     1.0000    0.4892    0.6570       554
          11     1.0000    1.0000    1.0000      1518
          12     0.8506    0.5266    0.6505      7018
          13     0.0142    0.1429    0.0258        28
          14     1.0000    0.5000    0.6

In [35]:
# CatBoost with Grid Search (Expanded, no l2_leaf_reg/border_count, fixed best score logic)
try:
    from catboost import CatBoostClassifier
    
    print("Starting CatBoost Grid Search (Expanded)...")
    
    # Expanded hyperparameter grid
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    print(f"Testing {len(params)} configurations...")
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'  # Use 'CPU' if GPU not available
            )
            
            cat_clf.fit(X_sup_train, y_sup_train)
            y_pred_cat = cat_clf.predict(X_sup_test)
            cat_f1 = f1_score(y_sup_test, y_pred_cat, average='macro')
            
            if cat_f1 > best_score:
                best_score = cat_f1
                best_params = {
                    'iterations': iterations,
                    'depth': depth,
                    'learning_rate': lr
                }
                best_model = cat_clf
                # Save the best model with unique name for fused features
                best_model_path = "best_catboost_classifier_fused.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\n" + "="*60)
    print("BEST CATBOOST HYPERPARAMETERS (Fused Features):")
    print(best_params)
    print(f"Best Macro F1: {best_score:.4f}")
    print("="*60)
    if best_model_path:
        print(f"\nBest CatBoost model saved to: {best_model_path}")
    
    # Final evaluation
    y_pred_best = best_model.predict(X_sup_test)
    print("\n" + "="*60)
    print("FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):")
    print("="*60)
    print(classification_report(y_sup_test, y_pred_best, digits=4))
    
except ImportError:
    print("CatBoost not installed. Install with: pip install catboost")

Starting CatBoost Grid Search (Expanded)...
Testing 27 configurations...


100%|██████████| 27/27 [15:57<00:00, 35.48s/it]


BEST CATBOOST HYPERPARAMETERS (Fused Features):
{'iterations': 500, 'depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.7804

Best CatBoost model saved to: best_catboost_classifier_fused.cbm

FINAL CLASSIFICATION REPORT (Best CatBoost - Fused):
              precision    recall  f1-score   support

           0     0.9814    0.9750    0.9782     99552
           1     1.0000    0.9995    0.9998      8524
           2     0.0249    0.5469    0.0476       128
           3     0.0110    0.1786    0.0207        56
           4     0.9991    0.9318    0.9643     65118
           5     0.9355    1.0000    0.9667       116
           6     1.0000    1.0000    1.0000     18324
           7     1.0000    0.9988    0.9994      1668
           8     0.9993    1.0000    0.9997     26008
           9     1.0000    1.0000    1.0000       856
          10     1.0000    1.0000    1.0000       554
          11     1.0000    1.0000    1.0000      1518
          12     0.6778    0.7395    0.7073      70

In [38]:
# Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES CLASSIFIER (Fused Features)")
print("="*60)

try:
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [10, 20, 30]

    params = list(itertools.product(n_estimators_grid, max_depth_grid))

    score = -1
    best_params = None
    best_model = None

    print(f"Testing {len(params)} configurations...")

    for n_est, depth in tqdm(params):
        try:
            et_clf = ExtraTreesClassifier(
                n_estimators=n_est,
                max_depth=depth,
                class_weight='balanced',
                random_state=13,
                n_jobs=-1
            )
            
            et_clf.fit(X_sup_train, y_sup_train)
            y_pred_et = et_clf.predict(X_sup_test)
            et_f1 = f1_score(y_sup_test, y_pred_et, average='macro')
            
            if et_f1 > score:
                score = et_f1
                best_params = {
                    'n_estimators': n_est,
                    'max_depth': depth
                }
                best_model = et_clf
                # Save best model
                with open('best_et_classifier_fused.pkl', 'wb') as f:
                    pickle.dump(et_clf, f)
        except Exception as e:
            print(f"\nError with params (n={n_est}, d={depth}): {str(e)}")
            continue

    if best_model is not None:
        print("\nBest Parameters:", best_params)
        print(f"Best Macro F1: {score:.4f}")
        print("Model saved to: best_et_classifier_fused.pkl\n")
        print(classification_report(y_sup_test, best_model.predict(X_sup_test), digits=4))
    else:
        print("\nAll configurations failed.")
        
except Exception as e:
    print(f"Unexpected error in Extra Trees: {str(e)}")

EXTRA TREES CLASSIFIER (Fused Features)
Testing 9 configurations...


100%|██████████| 9/9 [05:22<00:00, 35.80s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 10}
Best Macro F1: 0.7950
Model saved to: best_et_classifier_fused.pkl

              precision    recall  f1-score   support

           0     0.9816    0.9669    0.9742     99552
           1     0.9973    1.0000    0.9987      8524
           2     1.0000    0.1250    0.2222       128
           3     0.1951    0.1429    0.1649        56
           4     1.0000    0.9004    0.9476     65118
           5     0.8855    1.0000    0.9393       116
           6     0.9999    1.0000    0.9999     18324
           7     1.0000    1.0000    1.0000      1668
           8     0.9993    1.0000    0.9997     26008
           9     1.0000    1.0000    1.0000       856
          10     1.0000    1.0000    1.0000       554
          11     1.0000    1.0000    1.0000      1518
          12     0.6143    0.7415    0.6720      7018
          13     0.0033    0.7857    0.0066        28
          14     1.0000    1.0000    1.0000      5662

    accur

## Raw Features Classification (Comparison Baseline)

Now we'll train classifiers on **raw features only** (without graph embeddings) to compare the benefit of multimodal fusion:

In [39]:
# Prepare raw features (without embeddings)
# Use only the original numeric features from X_train/X_test

X_raw_train = X_train.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_train = y_train["Attack"].copy()

X_raw_test = X_test.drop(columns=["IPV4_SRC_ADDR", "IPV4_DST_ADDR", "h", "id"]).copy()
y_raw_test = y_test["Attack"].copy()

# Encode labels
y_raw_train_encoded = lab_enc.transform(y_train["Attack"])
y_raw_test_encoded = lab_enc.transform(y_test["Attack"])

# Compute sample weights
classes_raw = np.unique(y_raw_train_encoded)
class_w_raw = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_raw, y=y_raw_train_encoded
)
class_to_w_raw = {c: w for c, w in zip(classes_raw, class_w_raw)}
sample_weight_raw = pd.Series(y_raw_train_encoded).map(class_to_w_raw).values

# Feature names to str
X_raw_train.columns = X_raw_train.columns.map(str)
X_raw_test.columns = X_raw_test.columns.map(str)

print(f"Raw feature shape - Train: {X_raw_train.shape}, Test: {X_raw_test.shape}")
print(f"Number of classes: {len(np.unique(y_raw_train_encoded))}")

Raw feature shape - Train: (274519, 39), Test: (117651, 39)
Number of classes: 15


In [40]:
# Raw Features - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Raw Features Only)")

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = rf_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_raw.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

RANDOM FOREST (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [03:06<00:00, 20.76s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 30}
Best Macro F1: 0.7320
Model saved to: best_rf_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.9758    0.6785    0.8004     49862
           1     0.9998    0.9993    0.9995      4262
           2     0.0054    0.1562    0.0104        64
           3     0.0043    0.1429    0.0083        28
           4     0.9974    0.8941    0.9429     32559
           5     0.9508    1.0000    0.9748        58
           6     0.9995    0.9999    0.9997      9162
           7     1.0000    0.9940    0.9970       834
           8     0.9995    1.0000    0.9998     13004
           9     1.0000    1.0000    1.0000       428
          10     1.0000    0.9964    0.9982       277
          11     1.0000    1.0000    1.0000       759
          12     0.1431    0.7623    0.2409      3509
          13     0.0044    0.2143    0.0087        14
          14     0.9993    1.0000    0.9996      2831

    accurac

In [41]:
# Raw Features - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Raw Features Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='cuda'
        )
        
        xgb_clf.fit(X_raw_train, y_raw_train_encoded, sample_weight=sample_weight_raw)
        y_pred = xgb_clf.predict(X_raw_test)
        f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_raw.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_raw.json\n")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Raw Features Only)
Testing 27 configurations...


100%|██████████| 27/27 [05:22<00:00, 11.95s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.7370
Model saved to: best_xgb_classifier_raw.json

              precision    recall  f1-score   support

           0     0.9680    0.7717    0.8587     49862
           1     0.9998    0.9991    0.9994      4262
           2     0.0055    0.1875    0.0108        64
           3     0.0060    0.1786    0.0117        28
           4     0.9973    0.8876    0.9392     32559
           5     0.9508    1.0000    0.9748        58
           6     0.9996    0.9999    0.9997      9162
           7     0.9988    0.9976    0.9982       834
           8     0.9998    0.9998    0.9998     13004
           9     1.0000    1.0000    1.0000       428
          10     0.9928    1.0000    0.9964       277
          11     1.0000    1.0000    1.0000       759
          12     0.1644    0.6375    0.2614      3509
          13     0.0029    0.1429    0.0056        14
          14     0.9979    1.0000    0.998

In [42]:
# Raw Features - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Raw Features Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_raw_train, y_raw_train_encoded)
            y_pred = cat_clf.predict(X_raw_test)
            f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for raw features
                best_model_path = "best_catboost_classifier_raw.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Raw Features Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [05:03<00:00, 11.25s/it]



Best Parameters: {'iterations': 500, 'depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.7106

Best CatBoost model saved to: best_catboost_classifier_raw.cbm
              precision    recall  f1-score   support

           0     0.9662    0.7583    0.8498     49862
           1     0.9998    0.9991    0.9994      4262
           2     0.0044    0.2812    0.0086        64
           3     0.0037    0.2500    0.0073        28
           4     0.9976    0.7663    0.8668     32559
           5     0.5524    1.0000    0.7117        58
           6     0.9921    0.9953    0.9937      9162
           7     0.9811    0.9976    0.9893       834
           8     0.9912    0.9989    0.9951     13004
           9     1.0000    1.0000    1.0000       428
          10     0.9822    0.9964    0.9892       277
          11     1.0000    1.0000    1.0000       759
          12     0.1559    0.6227    0.2494      3509
          13     0.0018    0.2143    0.0036        14
          14     0.9919    0.9

In [43]:
# Raw Features - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Raw Features Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_raw_train, y_raw_train_encoded)
    y_pred = et_clf.predict(X_raw_test)
    f1 = f1_score(y_raw_test_encoded, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_raw.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_raw.pkl\n")
print(classification_report(y_raw_test_encoded, best_model.predict(X_raw_test), digits=4))

EXTRA TREES (Raw Features Only)
Testing 9 configurations...


100%|██████████| 9/9 [02:30<00:00, 16.68s/it]



Best Parameters: {'n_estimators': 100, 'max_depth': 30}
Best Macro F1: 0.7410
Model saved to: best_et_classifier_raw.pkl

              precision    recall  f1-score   support

           0     0.9656    0.8323    0.8940     49862
           1     1.0000    0.9993    0.9996      4262
           2     0.0052    0.2031    0.0101        64
           3     0.0038    0.1429    0.0075        28
           4     0.9974    0.8704    0.9296     32559
           5     0.9508    1.0000    0.9748        58
           6     0.9996    0.9999    0.9997      9162
           7     1.0000    0.9988    0.9994       834
           8     0.9998    1.0000    0.9999     13004
           9     1.0000    1.0000    1.0000       428
          10     1.0000    0.9964    0.9982       277
          11     1.0000    1.0000    1.0000       759
          12     0.1962    0.5811    0.2934      3509
          13     0.0043    0.2143    0.0084        14
          14     1.0000    1.0000    1.0000      2831

    accurac

## Embeddings Only Classification (Graph Features)

Now we'll train classifiers on **embeddings only** (graph features without raw features) to isolate the value of graph-based learning:

In [44]:
# Prepare embeddings-only features (graph features without raw data)
# Extract only the embedding columns (first 256 dimensions from graph encoder)

# From the fused dataframes, extract only embedding columns
# df_fuse_train has: [0-255] = embeddings, [256+] = raw features
num_embedding_dims = 256  # Based on the DGI encoder output dimension

X_emb_train = df_fuse_train.iloc[:, :num_embedding_dims].copy()
y_emb_train = df_fuse_train["Attacks"].copy()

X_emb_test = df_fuse_test.iloc[:, :num_embedding_dims].copy()
y_emb_test = df_fuse_test["Attacks"].copy()

# Compute sample weights
classes_emb = np.unique(y_emb_train)
class_w_emb = class_weight.compute_class_weight(
    class_weight="balanced", classes=classes_emb, y=y_emb_train
)
class_to_w_emb = {c: w for c, w in zip(classes_emb, class_w_emb)}
sample_weight_emb = y_emb_train.map(class_to_w_emb).values

# Feature names to str
X_emb_train.columns = X_emb_train.columns.map(str)
X_emb_test.columns = X_emb_test.columns.map(str)

print(f"Embeddings shape - Train: {X_emb_train.shape}, Test: {X_emb_test.shape}")
print(f"Number of classes: {len(np.unique(y_emb_train))}")
print(f"Using only graph embeddings (no raw features)")

Embeddings shape - Train: (548687, 256), Test: (235130, 256)
Number of classes: 15
Using only graph embeddings (no raw features)


In [45]:
# Embeddings Only - Random Forest
from sklearn.ensemble import RandomForestClassifier
import pickle

print("="*60)
print("RANDOM FOREST (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]
params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    rf_clf = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    rf_clf.fit(X_emb_train, y_emb_train)
    y_pred = rf_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = rf_clf
        # Save best model
        with open('best_rf_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(rf_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_rf_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

RANDOM FOREST (Embeddings Only)
Testing 9 configurations...


100%|██████████| 9/9 [14:14<00:00, 94.89s/it] 



Best Parameters: {'n_estimators': 200, 'max_depth': 10}
Best Macro F1: 0.6199
Model saved to: best_rf_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.9379    0.9635    0.9506     99552
           1     1.0000    0.8857    0.9394      8524
           2     0.0000    0.0000    0.0000       128
           3     1.0000    0.0179    0.0351        56
           4     0.9983    0.5990    0.7487     65118
           5     0.0012    0.0690    0.0024       116
           6     0.9999    1.0000    0.9999     18324
           7     1.0000    1.0000    1.0000      1668
           8     0.9993    1.0000    0.9997     26008
           9     0.5300    1.0000    0.6928       856
          10     1.0000    1.0000    1.0000       554
          11     1.0000    0.5000    0.6667      1518
          12     0.5543    0.6382    0.5933      7018
          13     0.0014    1.0000    0.0028        28
          14     1.0000    0.5000    0.6667      5662

    

In [46]:
# Embeddings Only - XGBoost
try:
    from xgboost import XGBClassifier
    
    print("="*60)
    print("XGBOOST (Embeddings Only)")
    print("="*60)
    
    n_estimators_grid = [100, 200, 300]
    max_depth_grid = [6, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(n_estimators_grid, max_depth_grid, learning_rate_grid))
    
    score = -1
    best_params = None
    best_model = None
    
    print(f"Testing {len(params)} configurations...")
    
    for n_est, depth, lr in tqdm(params):
        xgb_clf = XGBClassifier(
            n_estimators=n_est,
            max_depth=depth,
            learning_rate=lr,
            random_state=13,
            tree_method='hist',
            device='gpu'
        )
        
        xgb_clf.fit(X_emb_train, y_emb_train, sample_weight=sample_weight_emb)
        y_pred = xgb_clf.predict(X_emb_test)
        f1 = f1_score(y_emb_test, y_pred, average='macro')
        
        if f1 > score:
            score = f1
            best_params = {'n_estimators': n_est, 'max_depth': depth, 'learning_rate': lr}
            best_model = xgb_clf
            # Save best model
            xgb_clf.save_model('best_xgb_classifier_embeddings.json')
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {score:.4f}")
    print("Model saved to: best_xgb_classifier_embeddings.json\n")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("XGBoost not installed.")

XGBOOST (Embeddings Only)
Testing 27 configurations...


100%|██████████| 27/27 [27:05<00:00, 60.21s/it]



Best Parameters: {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.1}
Best Macro F1: 0.5433
Model saved to: best_xgb_classifier_embeddings.json

              precision    recall  f1-score   support

           0     0.8395    0.9885    0.9079     99552
           1     1.0000    1.0000    1.0000      8524
           2     0.0000    0.0000    0.0000       128
           3     0.0000    0.0000    0.0000        56
           4     0.9980    0.6514    0.7883     65118
           5     0.0012    0.1379    0.0025       116
           6     0.9998    1.0000    0.9999     18324
           7     1.0000    1.0000    1.0000      1668
           8     0.9992    0.5000    0.6665     26008
           9     0.0000    0.0000    0.0000       856
          10     1.0000    0.5000    0.6667       554
          11     0.6394    1.0000    0.7801      1518
          12     0.7823    0.5755    0.6632      7018
          13     0.0042    1.0000    0.0083        28
          14     1.0000    0.5000  

/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

In [47]:
# Embeddings Only - CatBoost (Expanded grid, no l2_leaf_reg/border_count)
try:
    from catboost import CatBoostClassifier
    
    print("="*60)
    print("CATBOOST (Embeddings Only, Expanded)")
    print("="*60)
    
    iterations_grid = [200, 300, 500]
    depth_grid = [4, 8, 10]
    learning_rate_grid = [0.01, 0.05, 0.1]
    
    params = list(itertools.product(iterations_grid, depth_grid, learning_rate_grid))
    
    best_score = -1
    best_params = None
    best_model = None
    best_model_path = None
    
    print(f"Testing {len(params)} configurations...")
    
    for iterations, depth, lr in tqdm(params):
        try:
            cat_clf = CatBoostClassifier(
                iterations=iterations,
                depth=depth,
                learning_rate=lr,
                auto_class_weights='Balanced',
                random_seed=13,
                verbose=False,
                task_type='GPU'
            )
            
            cat_clf.fit(X_emb_train, y_emb_train)
            y_pred = cat_clf.predict(X_emb_test)
            f1 = f1_score(y_emb_test, y_pred, average='macro')
            
            if f1 > best_score:
                best_score = f1
                best_params = {'iterations': iterations, 'depth': depth, 'learning_rate': lr}
                best_model = cat_clf
                # Save the best model with unique name for embeddings
                best_model_path = "best_catboost_classifier_embeddings.cbm"
                best_model.save_model(best_model_path)
        except Exception as e:
            print(f"\nError with params (it={iterations}, d={depth}, lr={lr}): {str(e)}")
            continue
    
    print("\nBest Parameters:", best_params)
    print(f"Best Macro F1: {best_score:.4f}\n")
    if best_model_path:
        print(f"Best CatBoost model saved to: {best_model_path}")
    print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))
    
except ImportError:
    print("CatBoost not installed.")

CATBOOST (Embeddings Only, Expanded)
Testing 27 configurations...


100%|██████████| 27/27 [19:05<00:00, 42.43s/it]



Best Parameters: {'iterations': 300, 'depth': 8, 'learning_rate': 0.1}
Best Macro F1: 0.6786

Best CatBoost model saved to: best_catboost_classifier_embeddings.cbm
              precision    recall  f1-score   support

           0     0.9831    0.9480    0.9652     99552
           1     0.9893    1.0000    0.9946      8524
           2     0.0000    0.0000    0.0000       128
           3     1.0000    0.0357    0.0690        56
           4     0.9981    0.3562    0.5250     65118
           5     0.0018    0.5603    0.0036       116
           6     0.9999    1.0000    0.9999     18324
           7     0.9994    1.0000    0.9997      1668
           8     0.9993    1.0000    0.9997     26008
           9     1.0000    1.0000    1.0000       856
          10     1.0000    1.0000    1.0000       554
          11     1.0000    1.0000    1.0000      1518
          12     0.5124    0.7639    0.6134      7018
          13     0.0042    1.0000    0.0083        28
          14     1.0000 

In [48]:
# Embeddings Only - Extra Trees
from sklearn.ensemble import ExtraTreesClassifier
import pickle

print("="*60)
print("EXTRA TREES (Embeddings Only)")
print("="*60)

n_estimators_grid = [100, 200, 300]
max_depth_grid = [10, 20, 30]

params = list(itertools.product(n_estimators_grid, max_depth_grid))

score = -1
best_params = None
best_model = None

print(f"Testing {len(params)} configurations...")

for n_est, depth in tqdm(params):
    et_clf = ExtraTreesClassifier(
        n_estimators=n_est,
        max_depth=depth,
        class_weight='balanced',
        random_state=13,
        n_jobs=-1
    )
    
    et_clf.fit(X_emb_train, y_emb_train)
    y_pred = et_clf.predict(X_emb_test)
    f1 = f1_score(y_emb_test, y_pred, average='macro')
    
    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est, 'max_depth': depth}
        best_model = et_clf
        # Save best model
        with open('best_et_classifier_embeddings.pkl', 'wb') as f:
            pickle.dump(et_clf, f)

print("\nBest Parameters:", best_params)
print(f"Best Macro F1: {score:.4f}")
print("Model saved to: best_et_classifier_embeddings.pkl\n")
print(classification_report(y_emb_test, best_model.predict(X_emb_test), digits=4))

EXTRA TREES (Embeddings Only)
Testing 9 configurations...


100%|██████████| 9/9 [04:44<00:00, 31.61s/it]



Best Parameters: {'n_estimators': 100, 'max_depth': 20}
Best Macro F1: 0.6813
Model saved to: best_et_classifier_embeddings.pkl

              precision    recall  f1-score   support

           0     0.9601    0.9899    0.9748     99552
           1     1.0000    0.9429    0.9706      8524
           2     0.0000    0.0000    0.0000       128
           3     0.0000    0.0000    0.0000        56
           4     0.9983    0.5002    0.6665     65118
           5     0.0018    0.3621    0.0036       116
           6     0.9999    1.0000    0.9999     18324
           7     1.0000    1.0000    1.0000      1668
           8     0.9993    1.0000    0.9997     26008
           9     1.0000    1.0000    1.0000       856
          10     1.0000    1.0000    1.0000       554
          11     1.0000    1.0000    1.0000      1518
          12     0.7768    0.4869    0.5986      7018
          13     0.0028    1.0000    0.0056        28
          14     1.0000    1.0000    1.0000      5662

    

/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/D/DesktopFiles/Hoc/DACN/paper_codes/Anomal-E/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

## Summary Comparison Table

Create a comparison table of all methods (Fused vs Raw features):

In [49]:
# Create a summary comparison table
# NOTE: After running all cells above, manually collect the F1 scores and create a comparison

print("="*95)
print("PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES")
print("="*95)
print("\nInstructions:")
print("1. Run all cells above to get F1 scores for each method")
print("2. Record the best Macro F1 score for each method")
print("3. Compare three approaches:")
print("   - Fused Features: Embeddings + Raw (Multimodal)")
print("   - Embeddings Only: Graph features only")
print("   - Raw Features: Traditional features only")
print("\nExpected format:")
print("-" * 95)
print(f"{'Method':<20} {'Fused (Emb+Raw)':<25} {'Embeddings Only':<25} {'Raw Only':<25}")
print("-" * 95)
print(f"{'Random Forest':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'XGBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'CatBoost':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print(f"{'Extra Trees':<20} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25} {'[INSERT SCORE]':<25}")
print("-" * 95)
print("\nAnalysis:")
print("- Fused features should show best performance (multimodal learning)")
print("- Embeddings capture structural/graph patterns")
print("- Raw features provide traditional statistical information")
print("- Compare to understand the contribution of each modality")

PERFORMANCE COMPARISON: FUSED vs EMBEDDINGS vs RAW FEATURES

Instructions:
1. Run all cells above to get F1 scores for each method
2. Record the best Macro F1 score for each method
3. Compare three approaches:
   - Fused Features: Embeddings + Raw (Multimodal)
   - Embeddings Only: Graph features only
   - Raw Features: Traditional features only

Expected format:
-----------------------------------------------------------------------------------------------
Method               Fused (Emb+Raw)           Embeddings Only           Raw Only                 
-----------------------------------------------------------------------------------------------
Random Forest        [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
XGBoost              [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
CatBoost             [INSERT SCORE]            [INSERT SCORE]            [INSERT SCORE]           
Extra Trees          [INSERT SCORE]           

## Key Insights & Interpretation

After running all experiments, analyze the results to answer:

1. **Which approach performs best overall?**
   - Fused features (multimodal) should ideally outperform single modalities
   - Compare the magnitude of improvements

2. **What is the value of graph embeddings?**
   - Compare Embeddings-only vs Raw-only to see if graph structure helps
   - If embeddings alone beat raw features, graph learning is beneficial

3. **Is multimodal fusion effective?**
   - Compare Fused vs (Embeddings + Raw separately)
   - Synergy should provide additional gains beyond individual modalities

4. **Which classifier is most suitable?**
   - Identify the best performing algorithm for each feature type
   - Consider computational cost vs accuracy trade-offs

5. **Feature contribution analysis:**
   - If Fused ≈ Embeddings > Raw: Graph structure dominates
   - If Fused ≈ Raw > Embeddings: Traditional features dominate
   - If Fused > Both: True synergy from multimodal learning